# TasteBud: GAN Based Recipe Generation with Graph

Introductory words about this project...

----
## Data Processing

Some words about source and process...

### Data Cleaning and Vectorizing

### Ingredient Graph

----
## Primary Model

### Ingredient Selector

### GAN RNN

### Training

----
## Baseline Model

----
## Results and Comparison